## Libraries, API Key

Don't forget to insert your Google API key

In [ ]:
import googlemaps
import pandas as pd
import numpy as np
import os

GOOGLE_API_KEY = ""
gmaps = googlemaps.Client(key = GOOGLE_API_KEY) 

np.random.seed(123)

## Constants

Import folder - the one with files like Leuven_aeds.csv (we have to make names more clear in final version)
Export folder - any folder you want

RATIO_EMERGENCY_NORMAL = 0.65 - this means that ambulance is 35% faster than a regular car for the same distance

In [ ]:
import_folder_path = "/Users/Jovan/Desktop/MDA_Project/DataSegmented"
export_folder_path = "/Users/Jovan/Desktop/MDA_Project/DistanceMatrices"

cities = ["Antwerpen", "Brugge", "Brussels", "Charleroi", "Gent", "Leuven", "Liege", "Oostende"]

RATIO_EMERGENCY_NORMAL = 0.65

In [ ]:
# def df_to_coordinates(df):
#     return list(df.itertuples(index=False, name=None))

In [ ]:
# def get_distance_matrix_chunk(orig, dest, mode, coef = 1)
#     result = gmaps.distance_matrix(origins = orig, destinations = dest, mode = mode)

#     time_matrix = [
#         [element['duration']['value'] for element in row['elements']]
#         for row in result['rows']
#     ]

#     return(time_matrix_df * coef)

In [ ]:
# def get_distance_matrix(origins, destinations, mode, coef=1):
#     num_origins = len(origins)
#     num_destinations = len(destinations)
#     result_matrix = np.zeros((num_origins, num_destinations))

#     orig_iter_list = range(0, num_origins, MAX_CHUNK)
#     dest_iter_list = range(0, num_destinations, MAX_CHUNK)
#     # Iterate over origins in chunks of 10
#     for i in orig_iter_list:
#         orig_chunk = origins[i:i+MAX_CHUNK]
        
#         for j in dest_iter_list:
#             dest_chunk = destinations[j:j+MAX_CHUNK]
            
#             # Calculate distance matrix for the chunk
#             chunk_matrix = get_distance_matrix_chunk(orig_chunk, dest_chunk, mode, coef)
#             # Update the result matrix with the chunk matrix
#             result_matrix[i:i+MAX_CHUNK, j:j+MAX_CHUNK] = chunk_matrix
    
#     # If there are remaining origins or destinations
#     if num_origins % MAX_CHUNK != 0 or num_destinations % MAX_CHUNK != 0:
#         remaining_matrix = get_distance_matrix_chunk(origins[num_origins - num_origins % MAX_CHUNK:], destinations[num_destinations - num_destinations % MAX_CHUNK:], mode, coef)
#         result_matrix[-num_origins % MAX_CHUNK:, -num_destinations % MAX_CHUNK:] = remaining_matrix
    
#     return result_matrix

## Functions

In [ ]:
# Adjusting coordinates so they suit directions() function

def format_coordinates(latitude, longitude):
    return f"{latitude}, {longitude}"

In [ ]:
# Calculates distance matrix for given sets of origin and destination coordinates

def get_distance_matrix(origins, destinations, mode, coef = 1):
    num_origins = len(origins)
    num_destinations = len(destinations)
    result_matrix = np.zeros((num_origins, num_destinations))

    for i in range(0, num_origins):
        for j in range(0, num_destinations):
            orig = format_coordinates(origins.iloc[i, 0], origins.iloc[i, 1])
            dest = format_coordinates(destinations.iloc[j, 0], destinations.iloc[j, 1])
            result_matrix[i][j] = gmaps.directions(orig, dest, mode)[0]['legs'][0]['duration']['value']

    return(result_matrix * coef)

## Individual (old) part - works 1 by 1. Use loop instead

### Import data

In [ ]:
os.chdir(import_folder_path)

aeds = pd.read_csv(city + "_aeds.csv")
cards = pd.read_csv(city + "_cards.csv")
vehicles = pd.read_csv(city + "_vehicles.csv")

### Moveable column

For AEDs: moveable is 1 if public, moveable is 0 if private or unknown
For vehicles: moveable is always 0

In [ ]:
aeds['public'] = aeds['public'].fillna(0)
aeds['public'] = aeds['public'].astype(bool)
aeds = aeds.rename(columns={'public': 'moveable'})

vehicles['moveable'] = 0

### Distance Matrix

In [ ]:
aeds_coordinates = aeds[['latitude', 'longitude']]
vehicles_coordinates = vehicles[['latitude', 'longitude']]
cards_coordinates = cards[['latitude', 'longitude']]

In [ ]:
aeds_dm = get_distance_matrix(aed_coordinates, cards_coordinates, "walking")
vehicles_dm = get_distance_matrix(vehicles_coordinates, cards_coordinates, "driving", RATIO_EMERGENCY_NORMAL)

In [ ]:
aeds_dm = np.hstack((aeds_dm, aeds["moveable"].values.reshape(-1, 1)))
vehicles_dm = np.hstack((vehicles_dm, vehicles["moveable"].values.reshape(-1, 1)))

In [ ]:
distance_matrix = np.vstack((aeds_dm, vehicles_dm))

### Export

In [ ]:
os.chdir(export_folder_path)

df = pd.DataFrame(distance_matrix)
df.to_csv("Leuven_distance_matrix.csv", index=False)

## Loop

In [ ]:
for city in cities:
    # Import data on the city
    os.chdir(import_folder_path)
    aeds = pd.read_csv(city + "_aeds.csv")
    cards = pd.read_csv(city + "_cards.csv")
    vehicles = pd.read_csv(city + "_vehicles.csv")
    print("Importing " + city + " successful!")

    # Make moveable columns for aeds and vehicles
    # For AEDs: moveable is 1 if public, moveable is 0 if private or unknown
    # For vehicles: moveable is always 0
    aeds['public'] = aeds['public'].fillna(0)
    aeds['public'] = aeds['public'].astype(bool)
    aeds = aeds.rename(columns={'public': 'moveable'})
    vehicles['moveable'] = 0
    print("Made moveable columns!")

    # Prepare distance matrix
    aeds_coordinates = aeds[['latitude', 'longitude']]
    vehicles_coordinates = vehicles[['latitude', 'longitude']]
    cards_coordinates = cards[['latitude', 'longitude']]
    
    aeds_dm = get_distance_matrix(aeds_coordinates, cards_coordinates, "walking")
    vehicles_dm = get_distance_matrix(vehicles_coordinates, cards_coordinates, "driving", RATIO_EMERGENCY_NORMAL)

    aeds_dm = np.hstack((aeds_dm, aeds["moveable"].values.reshape(-1, 1)))
    vehicles_dm = np.hstack((vehicles_dm, vehicles["moveable"].values.reshape(-1, 1)))
    distance_matrix = np.vstack((aeds_dm, vehicles_dm))

    # Export
    os.chdir(export_folder_path)
    df = pd.DataFrame(distance_matrix)
    df.to_csv(city + "_distance_matrix.csv", index=False)
